Estudo parametrico de um algoritmo imunológico em um problema de classificação

In [ ]:
Separar em 10 pastas de tamanho igual (15)

9 para treino e 1 para teste

10 execuções

Cada execução testo numa nova pasta (decremental)

Média e desvio padrão dos 10 testes

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install sklearn
# !pip install matplotlib

In [ ]:
import numpy as np
from numpy import random
import clonalg
import pandas as pd
from sklearn import preprocessing

Anticorpos treinados com exemplos de iris virginica [label 2]

In [ ]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

#Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)

#Train dataset
train = train.join(labels)
# O valor de train[4] define qual label será utilizada para o treino da população
train = train.loc[train[4] == 0]
train = train.drop(columns=[4])

Métodos de normalização além do StandartScaler deverão ser testados

In [ ]:
feature_num = len(train.axes[1])
feature_max = train.max()
feature_max  = feature_max.max()
feature_min = train.min()
feature_min  = feature_min.min()

Definição de hiperparâmetros

In [ ]:
population_size = 400
selection_size = 50
memory_set_percentage = 50

clone_rate = 40 ## Clone rate - 1 (Step 5 +-)
mutation_rate = 0.3
stop_codition = 100


d = 15 ## Individuos aleatorios - 2 (Step 2.5 +-)
sigma1 = 0.9
sigma2 = 0.3

Conversão de dataframe para numpy array

In [ ]:
train = train.to_numpy()

![](resources/V7zasui.png)

In [ ]:
stop = 0
population = clonalg.create_random_cells(population_size, feature_num, feature_min, feature_max)

while stop != stop_codition:
    # 1.
    for antigen in train:
        # 1.1
        population_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population]
        # 1.2
        population_affinity = sorted(population_affinity, key=lambda x: abs(x[1]))
        best_affinity = population_affinity[:selection_size]
        # 1.3
        clone_population = []
        for cell in best_affinity:
            cell_clones = clonalg.clone(cell, clone_rate)
            clone_population += cell_clones
        # 1.4 - 1.5
        mutaded_clone_population = []
        for cell in clone_population:
            mutated_clone = clonalg.hypermutate_variability(cell, mutation_rate, antigen)
            mutaded_clone_population.append(mutated_clone)
        # 1.6
        mutaded_clone_population.sort(key=lambda x: x[1])
        pop_size = round(len(clone_population)/100)*memory_set_percentage

        mutaded_clone_population = mutaded_clone_population[0:pop_size]

        # 1.7
        filtered_clone_population = list(filter(lambda x: x[1] < sigma2, mutaded_clone_population))

        # 1.8
        remaining_clone_population = clonalg.remove_similar_clones(filtered_clone_population, sigma1)

        # 1.9
        # Remova o atributo de afinidade das células em remaining_clone_population
        remaining_clone_population_no_affinity = [(cell[0],) for cell in remaining_clone_population]
        # Adicione remaining_clone_population_no_affinity à população
        population = population + remaining_clone_population_no_affinity

    # 2.0
    population = clonalg.suppress_similar_cells(population, sigma1)
        
    # 3
    new_cells = clonalg.create_random_cells(int(population_size * (d / 100)), feature_num, feature_min, feature_max)
    population += new_cells
    print("População: ", len(population), "     Iteração: ", stop)
    stop += 1

## Solução temporária para nested arrays nos indivíduos da população.

for i, cell in enumerate(population):
    # Verifica se a célula tem mais de uma dimensão
    if len(cell[0].shape) > 1:
        # Aplica numpy.ravel() para simplificar as dimensões
        flattened_cell = np.ravel(cell[0])
        population[i] = (flattened_cell,)
    else:
        population[i] = cell
        
    if isinstance(cell, tuple):
        array_cell = np.array(cell)
        flattened_cell = np.ravel(array_cell)
        population[i] = (flattened_cell)


In [ ]:
np.shape(population)

In [ ]:
for i, cell in enumerate(population):
    # Verifica se a célula tem mais de uma dimensão
    if len(cell[0].shape) > 1:
        # Aplica numpy.ravel() para simplificar as dimensões
        flattened_cell = np.ravel(cell[0])
        population[i] = (flattened_cell,)
    else:
        population[i] = cell
        
    if isinstance(cell, tuple):
        array_cell = np.array(cell)
        flattened_cell = np.ravel(array_cell)
        population[i] = (flattened_cell)

In [ ]:
antibodies = pd.DataFrame(population)
antibodies = pd.DataFrame(scaler.inverse_transform(antibodies), columns=antibodies.columns)
antibodies_array = antibodies.values

In [ ]:
antibodies

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.decomposition import PCA

# Extract the feature vectors from the population
feature_vectors = np.array([cell for cell in antibodies_array])


# Apply PCA to reduce the dimensionality to 3
pca = PCA(n_components=3)
reduced_population = pca.fit_transform(feature_vectors)

# Now, you can plot the reduced population using a 3D scatter plot

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = reduced_population[:, 0]
y = reduced_population[:, 1]
z = reduced_population[:, 2]

ax.scatter(x, y, z)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

feature_vectors = np.array([cell for cell in antibodies_array])

x = feature_vectors[:, 0]
y = feature_vectors[:, 1]
z = feature_vectors[:, 2]
c = feature_vectors[:, 3]

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

#Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(df)

#Train dataset
train = train.join(labels)
train = train.loc[train[4] == 2]
train = train.drop(columns=[4])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

train_array = train.values

feature_vectors = np.array([cell for cell in train_array])

x = feature_vectors[:, 0]
y = feature_vectors[:, 1]
z = feature_vectors[:, 2]
c = feature_vectors[:, 3]

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

#Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(df)

#Train dataset
train = train.join(labels)
train = train.loc[train[4] == 1]
train = train.drop(columns=[4])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

train_array = train.values

feature_vectors = np.array([cell for cell in train_array])

x = feature_vectors[:, 0]
y = feature_vectors[:, 1]
z = feature_vectors[:, 2]
c = feature_vectors[:, 3]

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

#Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(df)

#Train dataset
train = train.join(labels)
train = train.loc[train[4] == 0]
train = train.drop(columns=[4])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

train_array = train.values

feature_vectors = np.array([cell for cell in train_array])

x = feature_vectors[:, 0]
y = feature_vectors[:, 1]
z = feature_vectors[:, 2]
c = feature_vectors[:, 3]

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

In [ ]:
import pandas as pd
from sklearn import preprocessing
from train_loop import train_ais_classifier

In [ ]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

# Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
train = train.join(labels)

In [ ]:
feature_num = train.shape[1] - 1  # Menos a coluna de labels
feature_min = train.iloc[:, :-1].min().min()  # Menos a coluna de labels
feature_max = train.iloc[:, :-1].max().max()  # Menos a coluna de labels

# Treine o classificador AIS para cada classe (0, 1, 2) e armazene os resultados em uma lista
trained_populations = []
for class_label in range(3):
    train_class = train.loc[train[4] == class_label]
    train_class = train_class.drop(columns=[4])
    train_class = train_class.to_numpy()
    population = train_ais_classifier(train_class, feature_num, feature_min, feature_max)
    trained_populations.append(population)

In [ ]:
from sklearn.model_selection import train_test_split
import clonalg
from train_loop import multiclass_performance_measure

df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

# Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
train = train.join(labels)
# Dividir o conjunto de dados em treino e teste
train_data, test_data = train_test_split(train, test_size=0.3, random_state=42)

test_data_array = test_data.values
performance = multiclass_performance_measure(trained_populations, test_data_array)
print(f"Desempenho do classificador AIS: {performance * 100:.2f}%")


Multithread

In [4]:
from train_loop import train_clonalg_parallel
from train_loop import multiclass_performance_measure
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

# Standard Scaler
scaler = preprocessing.MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
train = train.join(labels)
# Dividir o conjunto de dados em treino e teste
train_data, test_data = train_test_split(train, test_size=0.3, random_state=42)

params = {
    'population_size': 300,
    'selection_size': 50,
    'memory_set_percentage': 50,
    'clone_rate': 15,
    'mutation_rate': 0.3,
    'stop_condition': 100,
    'd': 10,
    'sigma1': 0.9,
    'sigma2': 0.1,
    'feature_num': train_data.shape[1] - 1,
    'feature_min': train_data.min().min(),
    'feature_max': train_data.max().max(),
}

trained_populations = train_clonalg_parallel(train_data, params, [0, 1, 2])

# Avaliar o desempenho do classificador AIS
test_data_array = test_data.values
performance = multiclass_performance_measure(trained_populations, test_data_array)
print(f"Desempenho do classificador AIS: {performance * 100:.2f}%")

População:  87      Iteração:  0
População:  88      Iteração:  0
População:  103      Iteração:  0
Desempenho do classificador AIS: 28.89%


In [1]:
import numpy as np
from sklearn.model_selection import KFold
from train_loop import multiclass_performance_measure_v2
from train_loop import train_clonalg_parallel
from sklearn.metrics import confusion_matrix, classification_report
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

# Standard Scaler
scaler = preprocessing.MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
train = train.join(labels)

# Crie um objeto KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Armazenar as métricas de desempenho de cada iteração
performance_scores = []
# Armazenar todas as verdadeiras e previstas etiquetas através das divisões
all_true_labels = []
all_predicted_labels = []

train_data, test_data = train_test_split(train, test_size=0.3, random_state=42)

params = {
    'population_size': 400,
    'selection_size': 50,
    'memory_set_percentage': 50,
    'clone_rate': 15,
    'mutation_rate': 0.3,
    'stop_condition': 10,
    'd': 15,
    'sigma1': 0.9,
    'sigma2': 0.1,
    'feature_num': train_data.shape[1] - 1,
    'feature_min': train_data.min().min(),
    'feature_max': train_data.max().max(),
}

# Loop através das divisões de treinamento e teste
for i, (train_index, test_index) in enumerate(kfold.split(train), start=1):
    # Dividir o conjunto de dados em treinamento e teste com base nos índices
    train_data = train.iloc[train_index]
    test_data = train.iloc[test_index]

    # Treinar o classificador AIS
    trained_populations = train_clonalg_parallel(train_data, params, [0, 1, 2])

    # Avaliar o desempenho do classificador AIS
    test_data_array = test_data.values
    accuracy, true_labels, predicted_labels = multiclass_performance_measure_v2(trained_populations, test_data_array)
    performance_scores.append(accuracy)

    # Estender all_true_labels e all_predicted_labels com os rótulos verdadeiros e previstos da iteração atual
    all_true_labels.extend(true_labels)
    all_predicted_labels.extend(predicted_labels)

    # Imprimir o relatório de classificação para a iteração atual
    print("\n//////////////////////////////////////////////\n")
    print(f"Relatório de classificação para a pasta {i}:")
    print(classification_report(true_labels, predicted_labels))
    print("\n//////////////////////////////////////////////\n")

# Calcular a precisão média
print("\n//////////////////////////////////////////////\n")
mean_accuracy = np.mean(performance_scores)
print(f"Desempenho médio do classificador AIS: {mean_accuracy * 100:.2f}%")
print("\n//////////////////////////////////////////////\n")

# Calcular e exibir a matriz de confusão combinada
print("\n//////////////////////////////////////////////\n")
conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)
print("Matriz de confusão:")
print(conf_matrix)
print("\n//////////////////////////////////////////////\n")

# Calcular e exibir o relatório de classificação combinado
print("\n//////////////////////////////////////////////\n")
class_report = classification_report(all_true_labels, all_predicted_labels)
print("Relatório de classificação:")
print(class_report)
print("\n//////////////////////////////////////////////\n")


População:  101      Iteração:  0
População:  101      Iteração:  0
População:  96      Iteração:  0
População:  121      Iteração:  1
População:  126      Iteração:  1
População:  127      Iteração:  1
População:  133      Iteração:  2
População:  134      Iteração:  2
População:  147      Iteração:  2
População:  148      Iteração:  3
População:  138      Iteração:  3
População:  153      Iteração:  3
População:  147      Iteração:  4
População:  158      Iteração:  4
População:  148      Iteração:  4
População:  158      Iteração:  5
População:  151      Iteração:  5
População:  159      Iteração:  5
População:  157      Iteração:  6
População:  152      Iteração:  6
População:  150      Iteração:  6
População:  142      Iteração:  7
População:  152      Iteração:  7
População:  158      Iteração:  8
População:  155      Iteração:  7
População:  158      Iteração:  8
População:  148      Iteração:  9
População:  146      Iteração:  8
População:  152      Iteração:  9
População:  137